참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengege Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 05.007

### 문제에서 주어진 변수

#### 단면

W200$\times$100

In [ ]:
b_m, h_m, t_f_m, t_w_m, I_m4 = sy.symbols('b_m, h_m, t_f_m, t_w_m, I_m4', positive=True)

s_d = {
    b_m: 210e-3,
    h_m: 229e-3,
    t_f_m: 23.7e-3,
    t_w_m: 14.5e-3,
    I_m4: 113e6 * (1e-3**4),
}

#### 하중

In [ ]:
V_N = sy.Symbol('V_N')

s_d[V_N] = 80e3

### 전단응력

전단응력은 eq. 5.8 (p. 199) 과 같이 계산할 수 있다.

In [ ]:
Q_m3 = sy.Symbol('Q_m3', positive=True)
tau_Pa = V_N * Q_m3 / (t_w_m * I_m4)
tau_Pa

In [ ]:
tau_Pa.subs(s_d)

여기서 $Q$ 는 단면의 모멘트이다.

### (1) 웨브 web 에서의 최소 전단응력

중립면으로부터 멀어질 수록 전단력은 작아질 것이다. 웨브와 플랜지 flange 가 만나는 점에서 웨브의 전단응력은 최소일 것이다.

In [ ]:
y_min_m = h_m / 2 - t_f_m

최소 전단응력을 위한 $Q_{min}$ 는 플랜지 부분만 감안한다.

플랜지 면적

In [ ]:
A_f_m2 = b_m * t_f_m

플랜지의 도심 거리

In [ ]:
y_bar_min_m = h_m/2 - (t_f_m/2)

$Q_{min}$

In [ ]:
Q_min_m3 = A_f_m2 * y_bar_min_m
Q_min_m3

값을 확인해 본다.

In [ ]:
Q_min_m3.subs(s_d)

$mm^3$ 단위로도 확인해본다.

In [ ]:
Q_min_m3.subs(s_d)*(1e3**3)

응력 식의 $Q$에 $Q_{min}$을 대입하면 웨브의 최소 전단응력은 다음과 같다.

In [ ]:
tau_min_Pa = tau_Pa.subs({Q_m3: Q_min_m3})
tau_min_Pa

값을 확인한다.

In [ ]:
tau_min_Pa.subs(s_d)

$MPa$ 단위 값을 확인한다.

In [ ]:
tau_min_Pa.subs(s_d)*1e-6

### (2) 웨브에서의 최대 전단응력

중립면에서 최대가 될 것이다.

In [ ]:
y_max_m = 0

최대 전단응력을 위한 $Q_{max}$ 는 플랜지 부분과 웨브의 위쪽 절반을 감안한다.

웨브 위쪽 절반 면적은 다음과 같다.

In [ ]:
A_w_half_m2 = t_w_m * (h_m/2 - t_f_m)
A_w_half_m2

웨브 위쪽 절반의 도심은 아래와 같을 것이다.

In [ ]:
y_w_max_m = (h_m/2 - t_f_m) / 2
y_w_max_m

$Q_{max}$ 는 플랜지 부분의 $Q$ 와 웨브 위쪽 절반의 $Q$를 합한 것이다.

In [ ]:
Q_max_m3 = A_w_half_m2 * y_w_max_m + A_f_m2 * y_bar_min_m
Q_max_m3

In [ ]:
sy.simplify(Q_max_m3)

$Q$ 값을 확인한다.

In [ ]:
Q_max_m3.subs(s_d)

최대 전단응력을 구한다.

In [ ]:
tau_max_Pa = sy.simplify(tau_Pa.subs(Q_m3, Q_max_m3))
tau_max_Pa

최대 전단응력 값을 확인한다.

In [ ]:
tau_max_Pa.subs(s_d)

$MPa$ 단위로 최대 전단응력 값을 확인한다.

In [ ]:
tau_max_Pa.subs(s_d) * 1e-6

### (3) 웨브가 감당하는 전단력의 백분률

웨브의 $y$ 를 따라 전단응력은 포물선 형태를 가진다.

선도를 그려서 적분하는 것도 가능하지만 이번에는; 면적을 밑변 $h$, 높이 $m$로 표시하면 다음과 같다.

In [ ]:
h, m, a = sy.symbols('h, m, a', positive=True)
y_m = sy.symbols('y_m', real=True)

$$p(y)=m - a y^2 $$

위 포물선은 $y=0$ 에서 최대값 $m$ 이다.  $y= \pm \frac {h}{2}$ 지점에서 0으로 만드는 $a$ 를 다음과 같이 구한다.

In [ ]:
a_sol_list = sy.solve(m - (a * y_m**2).subs(y_m, h/2), a)
a_sol_list

In [ ]:
a = a_sol_list[0]

따라서 적분값은 다음과 같을 것이다.

In [ ]:
a_para = sy.integrate(m - (a * y_m**2), (y_m, -h/2, h/2))
a_para

웨브가 담당하는 전단력은 다음과 같다.

$$
V_{web} = t_{web} \left[A_{para} + A_{rect} \right]
$$

$$
A_{para} = \frac{2}{3} h_{web} \cdot (\tau_{max}-\tau_{min})
$$

$$h_{web}=h - 2 t_{flange}$$

$$
A_{rect} = h_{web} \cdot \tau_{min}
$$

$$
\begin{align}
    V_{web} &= t_{web} \left[\frac{2}{3} h_{web} \cdot (\tau_{max}-\tau_{min}) + h_{web} \cdot \tau_{min} \right] \\
    & = t_{web} h_{web} \left[\frac{2}{3} (\tau_{max}-\tau_{min}) + \tau_{min} \right]
\end{align}
$$

In [ ]:
h_w_m = h_m - 2 * t_f_m

In [ ]:
A_para_N_m = 2* h_w_m * (tau_max_Pa - tau_min_Pa)/3
sy.simplify(A_para_N_m)

In [ ]:
A_rect_N_m = h_w_m * tau_min_Pa
sy.simplify(A_rect_N_m)

In [ ]:
V_w_N = t_w_m * (A_para_N_m + A_rect_N_m)
sy.simplify(V_w_N)

웨브 부분이 담당하는 전단력은 다음과 같다.

In [ ]:
V_w_N.subs(s_d)

전체 전단력 대비 백분율은 다음과 같다.

In [ ]:
(V_w_N/V_N).subs(s_d) * 100

### 전단응력의 분포<br>Distribution of shear stress

`y` 좌표 배열 준비<br>`y` coordinate array

In [ ]:
dy_m = 1e-3
dy_half_m = dy_m * 0.5
y_m_array = np.arange(s_d[h_m] * (-0.5), s_d[h_m] * (0.5) + dy_half_m, dy_m)

Q 값을 계산하기 위해 중립면에서 `y` 만큼 떨어진 넓이를 계산.<br>Calculate area whose distance from the neutral axis is `y`. 

In [ ]:
A_m2_array = s_d[b_m] * (s_d[h_m] * 0.5 - abs(y_m_array))

`y` 와 그 위 면적을 그림으로 표시.<br>Plot `y` vs area above it.

In [ ]:
plt.plot(A_m2_array, y_m_array)
plt.xlim(xmin=0)
plt.xlabel('A(m^2)')
plt.ylabel('y(m)')
plt.grid(True)
plt.show()

`y`와 `y` 위 면적의 도심의 위치를 그림으로 표시.<br>Plot `y` vs centroid of area above `y`.

In [ ]:
y_bar_m_array = s_d[h_m] * 0.5 - abs(y_m_array) * 0.5 + abs(y_m_array)

In [ ]:
plt.plot(y_bar_m_array, y_m_array)
plt.xlabel('$\overline{y}(m^2)$')
plt.ylabel('y(m)')
plt.xlim(xmin=0)
plt.grid(True)
plt.show()

Q 값을 계산<br>Calculate Q

In [ ]:
Q_m3_array = (A_m2_array * y_bar_m_array)

In [ ]:
plt.plot(Q_m3_array, y_m_array)
plt.xlabel('$Q(m^3)$')
plt.ylabel('y(m)')
plt.xlim(xmin=0)
plt.grid(True)
plt.show()

전단응력을 계산함<br>Calculate the shear stress

$$
\tau=\frac{VQ}{IB}
$$

In [ ]:
tau_Pa_array = (s_d[V_N]) * Q_m3_array / float((s_d[b_m] * I_m4.subs(s_d)))

In [ ]:
plt.plot(tau_Pa_array, y_m_array)
plt.xlabel('$\sigma$(Pa)')
plt.ylabel('y(m)')
plt.xlim(xmin=0)
plt.grid(True)
plt.show()